In [1]:
import chromedriver_autoinstaller
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
import datetime as dt

In [2]:
def rip_11st(x):
    prod_data_list = []

    for i in x:
        prod_data = {}

        try:
            prod_data['상품명'] = i.find_element(By.CSS_SELECTOR, 'p.info_tit a').text
        except:
            continue

        try:
            prod_data['판매가'] = int((i.find_element(By.CSS_SELECTOR, 'strong.sale_price').text).replace(",", ""))
        except:
            prod_data['판매가'] = None

        try:
            prod_data['판매몰'] = i.find_element(By.CSS_SELECTOR, 'p.benefit_tit').text
        except:
            prod_data['판매몰'] = None

        try:
            prod_data['이미지'] = i.find_element(By.CSS_SELECTOR, 'div.photo_wrap img').get_attribute('src')
        except:
            prod_data['이미지'] = None

        prod_data_list.append(prod_data)
    
    return prod_data_list

In [59]:
def extract_11st(n, url):
    driver = webdriver.Chrome()
    driver.get(url)
    driver.implicitly_wait(3)

    prod_data_all = []
    idx = 1

    while n >= idx:
        for i in range(0, 5):
            driver.execute_script("window.scrollTo(0, (document.body.scrollHeight)*0.8);")
            time.sleep(0.3)

        prod_list = driver.find_elements(By.CSS_SELECTOR, 'div.normal_prd div.total_listitem')
        prod_data_all += rip_11st(prod_list)
        idx += 1

        try:
            btn = "div#list_paging a[data-log-body=\"{'btn_name':'%d'}\"]"
            button = driver.find_element(By.CSS_SELECTOR, btn % idx)
            button.click()
        except:
            btn = "div#list_paging a.next"
            button = driver.find_element(By.CSS_SELECTOR, btn)
            button.click()
    
    return prod_data_all

In [4]:
url = 'https://www.11st.co.kr/category/DisplayCategory.tmall?method=getDisplayCategory2Depth&dispCtgrNo=1001704'

In [63]:
shoes = extract_11st(30, url)

In [66]:
pd.DataFrame(shoes).to_csv('shoes.csv', index = False)